**To see the traning process please refer to [Traing the model for classification](https://www.kaggle.com/sd4321/latest-fastai-training-inference-prediction)****

In [ ]:
import pandas as pd
import fastai
from fastai.vision.all import *

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
image_path = Path('../input/cassava-leaf-disease-classification/train_images')
training_data_file = Path('../input/cassava-leaf-disease-classification/train.csv')
sample_submission_file = Path('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
train_df = pd.read_csv(training_data_file)

In [ ]:
src_size = (PILImage.create('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')).size
bs = 16

def get_x(r):return image_path/r['image_id']

def get_y(r):return r['label']

item_tfms = [Resize(src_size)]

batch_tfms = [RandomResizedCrop(size=224), *aug_transforms(mult=1.0, do_flip=True, max_rotate=40.0),Normalize.from_stats(*imagenet_stats)]


datablockk = DataBlock(blocks=(ImageBlock, CategoryBlock),splitter=RandomSplitter(seed=42),get_x=get_x,get_y=get_y,item_tfms = item_tfms,batch_tfms = batch_tfms )

image_data_loader = datablockk.dataloaders(train_df, bs=bs)

In [ ]:
learn = load_learner('../input/trainedmodel/export (1).pkl')

In [ ]:
test_image_files = Path('../input/cassava-leaf-disease-classification/test_images')
image_data_loader_test = image_data_loader.test_dl(get_image_files(test_image_files))
labels = []
files = []
for file in list(test_image_files.glob("*.jpg")):
    files.append(file.name)
    labels.append(str(learn.predict(file)[0]))
sub = pd.read_csv(sample_submission_file)
sub.head()
sub['label'] = labels
sub.to_csv('submission.csv', index=False)